In [41]:
import boto3
import re
import datetime
from io import BytesIO
import os
from PIL import Image

# Clase OCR

In [26]:
class TextractOCR:
    def __init__(self, aws_region="us-east-1"):
        self.textract = boto3.client("textract", region_name=aws_region)

    def extract_text_from_file(self, file_path):
        """Extrae texto plano línea por línea desde una imagen"""
        with open(file_path, "rb") as image_file:
            image_bytes = image_file.read()

        response = self.textract.detect_document_text(Document={'Bytes': image_bytes})
        
        raw_text = ""
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                raw_text += item["Text"] + "\n"

        return raw_text
    

    def extract_text_from_image(self, i):
        image = Image.open(BytesIO(i))
        buffer = BytesIO()
        image.save(buffer, format="PNG")  # Asegúrate que sea PNG si ese es el formato real
        image_bytes = buffer.getvalue()

        response = self.textract.detect_document_text(Document={'Bytes': image_bytes})

        raw_text = ""
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                raw_text += item["Text"] + "\n"

        return raw_text


    def extract_and_save(self, file_path, output_txt_path):
        """Extrae texto y lo guarda en un archivo .txt"""
        text = self.extract_text_from_file(file_path)
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(text)
        return text


# Implementación OCR

In [3]:
ocr = TextractOCR()

In [4]:
raw_text = ocr.extract_text_from_file("../Cadena/FT Procesada/FT1.png")

In [29]:
print(raw_text)

7
0000000 000 0000
*0000000*
LO
ERÍA
8
DIST: ZZZZZ
SORTEO 3097
**Cruz Roja Colombiana
Ganas Ayudando
COD. Y NOMBRE DISTRIBUIDOR
Juega el martes 15 de Abril de
DIRECCIÓN
2025 en Bogotá D.C. a las 10:55
SORTEO 3097
13
00-0000
CIUDAD (DEPARTAMENTO)
000000000000
TELÉFONO
p.m. por el canal 1y facebook live.
NÚMERO2023
0000000
Fracción $7.500 Billete $15.000
Hh09ñ)?
7
SERIE 244
2
4
6
SORTEO 3097
FRACCIÓN1$7.500
BILLETE $15.000
2
+
LO
ERÍA
6
5
5
JUEGA EL 15 ABRIL
Cruz Roja
""Cruz Roja Colombiana
10:55 p.m.porCandllyFaccbooktLive
Colombiana
cadena. 4
10
Ganas Ayudando
1
La
JUEGA ELMARTES15 DE ABRIL 2025 EN BOGOTA D.C.A LAS 10:55 P.M. POR ELCANAL1Y FACEBOOK LIVE
alud
7.000
MILLONES
nuestra mayor riqueza
3
premio mayor
6
1
Día mundial de la salud 07 abril
SERIE 244
DOSCIENTOS CUARENTA Y CUATRO
NUMERO
3
9
2023
4
VEINTE
8
VEINTITRES
2
1
2
+
LO+ERÍA
SORTEO 3097
Cruz Roja
**Cruz Reja Colombiana
Colombiana
Ganas Ayudando
FRACCIÓN $7.500
cadena. 4
BILLETE $15.000
JUEGA EL15 ABRIL
10.55 p.m.porCanallyF

# Clase Validaciones

In [36]:

class Validaciones:
    def __init__(self, raw_text: str, sorteo: str, fecha_sorteo: str, dia_de_juego: str, hora_de_juego: str, premio_mayor: str, valor_billete: str, valor_fraccion: str, serie: str, numero: str, numero_escrito: list):
        self.text = self._preprocesar(raw_text)
        self.sorteo = sorteo.lower()
        self.fecha_sorteo = fecha_sorteo.lower()
        self.dia_de_juego = dia_de_juego.lower()
        self.hora_de_juego = hora_de_juego.lower()
        self.premio_mayor = premio_mayor.lower()
        self.valor_billete = valor_billete.lower()
        self.valor_fraccion = valor_fraccion.lower()
        self.serie = serie.lower()
        self.numero = numero.lower()
        self.numero_escrito = [p.lower().strip() for p in numero_escrito if p.strip()]
    
    def _preprocesar(self, texto: str) -> str:
        # Convierte a minúsculas, elimina saltos de línea y múltiples espacios
        texto = texto.lower()
        texto = texto.replace("\n", " ")
        texto = re.sub(r"\s+", " ", texto)  # Remueve espacios duplicados
        return texto    

    def check_sorteo(self):
        return self.sorteo in self.text
    
    def check_fecha(self):
        # Acepta variantes como "15 de abril", "abril 15", etc.
        fecha_variantes = [
            self.fecha_sorteo,
            re.sub(r"(\d{1,2}) de (\w+)", r"\2 \1", self.fecha_sorteo),  # "15 de abril" -> "abril 15"
        ]
        return any(var in self.text for var in fecha_variantes)

    def check_dia(self):
        return self.dia_de_juego in self.text

    def check_hora(self):
        return self.hora_de_juego in self.text

    def check_premio_mayor(self):
        return self.premio_mayor in self.text

    def check_valor_billete(self):
        return self.valor_billete in self.text

    def check_valor_fraccion(self):
        return self.valor_fraccion in self.text

    def check_serie(self):
        return self.serie in self.text
    
    def check_numero(self):
        return self.numero in self.text
    
    def check_numero_escrito(self):
        return all(palabra in self.text for palabra in self.numero_escrito)

    def run_all_checks(self):
        return {
            "sorteo": self.check_sorteo(),
            "fecha": self.check_fecha(),
            "dia": self.check_dia(),
            "hora": self.check_hora(),
            "premio_mayor": self.check_premio_mayor(),
            "valor_billete": self.check_valor_billete(),
            "valor_fraccion": self.check_valor_fraccion(),
            "serie": self.check_serie(),
            "numero": self.check_numero(),
            "numero_escrito": self.check_numero_escrito()
        }
    
    def count_sorteo(self):
        return self.text.count(self.sorteo)

    def count_fecha(self):
        variantes = set([
            self.fecha_sorteo,
            re.sub(r"(\d{1,2}) de (\w+)", r"\2 \1", self.fecha_sorteo),  # "15 de abril" → "abril 15"
            re.sub(r"(\d{1,2})/(\d{1,2})/(\d{4})", r"\1/\2", self.fecha_sorteo),  # "15/04/2025" → "15/04"
        ])
        return sum(self.text.count(var) for var in variantes if var)

    def count_dia(self):
        return self.text.count(self.dia_de_juego)

    def count_hora(self):
        return self.text.count(self.hora_de_juego)

    def count_premio_mayor(self):
        return self.text.count(self.premio_mayor)

    def count_valor_billete(self):
        return self.text.count(self.valor_billete)

    def count_valor_fraccion(self):
        return self.text.count(self.valor_fraccion)

    def count_serie(self):
        return self.text.count(self.serie)
    
    def count_numero(self):
        return self.text.count(self.numero)

    def count_numero_escrito(self):
        if len(self.numero_escrito) == 1:
            return self.text.count(self.numero_escrito[0])
        
        palabras_texto = self.text.split()
        count = 0
        window_size = len(self.numero_escrito) + 3  # Puedes ajustar la ventana según precisión/difusión

        for i in range(len(palabras_texto) - window_size + 1):
            ventana = palabras_texto[i:i + window_size]
            if all(palabra in ventana for palabra in self.numero_escrito):
                count += 1

        return count

    def run_all_counts(self):
        return {
            "sorteo": self.count_sorteo(),
            "fecha": self.count_fecha(),
            "dia": self.count_dia(),
            "hora": self.count_hora(),
            "premio_mayor": self.count_premio_mayor(),
            "valor_billete": self.count_valor_billete(),
            "valor_fraccion": self.count_valor_fraccion(),
            "serie": self.count_serie(),
            "numero": self.count_numero(),
            "numero_escrito": self.count_numero_escrito()
        }


# Implementación Validaciones

In [56]:
validador = Validaciones(
    raw_text=raw_text,
    sorteo="3097",
    fecha_sorteo="15 de Abril",
    dia_de_juego="martes",
    hora_de_juego="10:55 p.m.",
    premio_mayor="7.000",
    valor_billete="$15.000",
    valor_fraccion="$7.500",
    serie="244",
    numero="2023",
    numero_escrito=["veinte", "veintitres"]
)

resultados = validador.run_all_checks()
counts = validador.run_all_counts()

In [57]:
print("Resultados de las validaciones:")
for key, value in resultados.items():
    print(f"{key}: {value}")


print("\nCounts:")
for key, value in counts.items():
    print(f"{key} count: {value}")

Resultados de las validaciones:
sorteo: True
fecha: True
dia: True
hora: True
premio_mayor: True
valor_billete: True
valor_fraccion: True
serie: True
numero: True
numero_escrito: True

Counts:
sorteo count: 4
fecha count: 2
dia count: 3
hora count: 3
premio_mayor count: 1
valor_billete count: 3
valor_fraccion count: 3
serie count: 3
numero count: 3
numero_escrito count: 4
